In [1]:
# install missing packages
!conda install -y -c bioconda mysqlclient


Solving environment: done

# All requested packages already installed.



In [11]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
from getpass import getpass

# define user parameters
db_name = 'test_db'
table_name = 'test_table'
server = 'localhost'
port = '3306'
user = 'root'
infile = r'test.csv'
db = db_name
db_tbl_name = table_name

# get password for SQL
pw = getpass('Enter your SQL password')

def csv_to_df(infile, headers = []):
    '''
    Load a csv file into a dataframe; if csv does not have headers, use the headers arg to create a list of headers; rename unnamed columns to conform to mysql column requirements
    '''
    if len(headers) == 0:
        df = pd.read_csv(infile)
    else:
        df = pd.read_csv(infile, header = None)
        df.columns = headers
    for r in range(10):
        try:
            df.rename( columns={'Unnamed: {0}'.format(r):'Unnamed{0}'.format(r)},    inplace=True )
        except:
            pass
    return df


def dtype_mapping():
    '''
    Create a mapping of df dtypes to mysql data types (not perfect, but close enough)
    '''
    return {'object' : 'TEXT',
        'int64' : 'INT',
        'float64' : 'FLOAT',
        'datetime64' : 'DATETIME',
        'bool' : 'TINYINT',
        'category' : 'TEXT',
        'timedelta[ns]' : 'TEXT'}

def mysql_engine(user = user, password = pw, host = server, port = port, database = db_name):
    '''
    Create a sqlalchemy engine
    '''
    engine = create_engine("mysql://{0}:{1}@{2}:{3}/{4}?charset=utf8".format(user, password, host, port, database))
    return engine


def mysql_conn(engine):
    '''
    Create a mysql connection from sqlalchemy engine
    '''
    conn = engine.raw_connection()
    return conn

def gen_tbl_cols_sql(df):
    '''
    Create sql input for table names and types
    '''
    dmap = dtype_mapping()
    sql = "pi_db_uid INT AUTO_INCREMENT PRIMARY KEY"
    
    # remove space in col names and replace / with _
    oldcolnames = list(df.columns)
    newcolnames = ["".join(k.split(' ')) for k in oldcolnames]
    newcolnames = [k.replace('/', '_') for k in newcolnames]
    col_rename_dict = {i:j for i,j in zip(oldcolnames,newcolnames)}
    df.rename(columns=col_rename_dict, inplace=True)
    # replace empty names
    df1 = df.rename(columns = {"" : "nocolname"})
    
    hdrs = df1.dtypes.index
    hdrs_list = [(hdr, str(df1[hdr].dtype)) for hdr in hdrs]
    for i, hl in enumerate(hdrs_list):
        sql += " ,{0} {1}".format(hl[0], dmap[hl[1]])
    return sql


def create_mysql_tbl_schema(df, conn, db, tbl_name):
    '''
    Create a mysql table from a df
    '''
    tbl_cols_sql = gen_tbl_cols_sql(df)
    sql = "USE {0}; CREATE TABLE IF NOT EXISTS {1} ({2})".format(db, tbl_name, tbl_cols_sql)
    cur = conn.cursor()
    cur.execute(sql)
    cur.close()
    conn.commit()


def df_to_mysql(df, engine, tbl_name, if_exists_condition):
    '''
    Write df data to newly create mysql table
    '''
    df.to_sql(tbl_name, engine, if_exists=if_exists_condition)


Enter your SQL password········


In [10]:
# main code

# load data
df = csv_to_df(infile)

# create schema and table
create_mysql_tbl_schema(df, mysql_conn(mysql_engine()), db, db_tbl_name)

# write pandas DF to SQL table
df_to_mysql(df, mysql_engine(), db_tbl_name, if_exists_condition = 'replace') # replace or append
